In [1]:
import sys
sys.path.append("..")

import datetime as dt
from ipywidgets import interact
import itertools
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import seaborn as sns
import statsmodels.api as sm
import tqdm
import vectorbtpro as vbt

import legitindicators as li
import pandas_ta as pta

from lib import utils, indicators, binance_loader

In [391]:
df = binance_loader.load_ohlc_in_date_range('SOLUSDT', '2022-01-01', '2022-12-31')

In [384]:
freq = '15min'
df = df.resample(freq).agg({'open':'first', 'high':'max', 'low':'min', 'close':'last', 'volume':'sum', 'quoteVolume':'sum', 'nTrades':'sum', 'upVolume':'sum', 'upQuoteVolume':'sum'})

In [392]:
for lag in range(1, 60, 2):
    df[f'y_return_{lag:02d}'] = df['close'].pct_change(lag).shift(-lag)

In [398]:
df['volume_delta'] = 2*df['upVolume'] - df['volume']
for l in range(1, 60, 2):
    df[f'vd_rolling_corr_{l:02d}'] = df['volume_delta'].rolling(l).corr(df[f'y_return_{lag:02d}'])
    df[f'x_volume_delta_{l:02d}'] = df['volume_delta'].rolling(l).sum() #* df[f'vd_rolling_corr_{l:02d}'].clip(0, 1)

In [399]:
def get_corrs(_df, pfx1, pfx2):
    s1 = utils.get_prefixed_cols(_df, pfx1)
    s2 = utils.get_prefixed_cols(_df, pfx2)
    corrs = {}
    for x in itertools.product(s1, s2):
        corrs.update({x: _df[x[0]].corr(_df[x[1]])})
    corrs = pd.Series(corrs).unstack()
    return corrs

In [400]:
corrs = get_corrs(df, 'x_volume_delta_', 'y_')

In [401]:
from sklearn.model_selection import KFold

In [ ]:
kf = KFold(4)

for i, (train_index, test_index) in enumerate(kf.split(df)):
    sdf = df.iloc[test_index]
    corrs = get_corrs(sdf, 'x_volume_delta_', 'y_')
    fig, ax = plt.subplots()
    sns.heatmap(corrs, ax=ax)

In [ ]:
fig = go.FigureWidget(make_subplots(rows=3, cols=1, shared_xaxes=True, row_heights=[0.6, 0.2, 0.2]))
fig.add_trace(go.Candlestick(), row=1, col=1)
fig.add_trace(go.Scatter(), row=2, col=1)
fig.add_trace(go.Scatter(), row=3, col=1)
fig.update_layout(height=600, margin=dict(l=20,r=20,b=20,t=20), xaxis=dict(rangeslider=dict(visible=False)))

@interact(date=np.unique(df.index.date), col=df.columns, col2=df.columns)
def update(date, col, col2):
   with fig.batch_update():
      _sdf = df.loc[str(date)]
      fig.data[0].x, fig.data[0].open, fig.data[0].high = _sdf.index, _sdf['open'], _sdf['high']
      fig.data[0].low, fig.data[0].close = _sdf['low'], _sdf['close']
      fig.data[1].x, fig.data[1].y = _sdf.index, _sdf[col]
      fig.data[2].x, fig.data[2].y = _sdf.index, _sdf[col2]
      fig.update_layout()
fig
